In [1]:
# AE502_HW4_IOD 
# Joshua Yuan
# This program uses orekit to first determine the positions
# of the satellite based on the observations from the ground
# Then the initial orbit is found and refined using the
# Gauss's initial orbit determination method

# packages
from math import radians, pi, degrees, sin, cos, sqrt, acos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# The ORbite Extrapolation KIT (OREKIT) is the main package used
#initialize orekit and JVM
import orekit
orekit.initVM()

from orekit.pyhelpers import setup_orekit_curdir, absolutedate_to_datetime
setup_orekit_curdir()

from org.orekit.frames import FramesFactory, TopocentricFrame
from org.orekit.estimation.measurements import GroundStation, ObservableSatellite
from org.orekit.estimation.measurements import AngularRaDec, AngularAzEl
from org.orekit.bodies import OneAxisEllipsoid, GeodeticPoint, CelestialBodyFactory
from org.orekit.estimation.iod import IodLambert, IodGibbs, IodLaplace, IodGooding
from org.orekit.time import TimeScalesFactory, AbsoluteDate, DateComponents, TimeComponents
from org.orekit.utils import IERSConventions, Constants, PVCoordinates, PVCoordinatesProvider, AbsolutePVCoordinates
from org.hipparchus.geometry.euclidean.threed import Vector3D, SphericalCoordinates
from org.orekit.data import DataProvidersManager, ZipJarCrawler
from org.orekit.orbits import KeplerianOrbit, CartesianOrbit, PositionAngle, OrbitType
from org.orekit.propagation.analytical.tle import TLE, TLEPropagator
from org.orekit.propagation import SpacecraftState
from org.orekit.models.earth import ReferenceEllipsoid

In [2]:
TS = TimeScalesFactory.getTimeScales() # intitialize time scales
MJD = TS.getModifiedJulianEpoch() # get the initial time for MJD

FF = FramesFactory.getFrames() #initialize reference frames
ECI = FF.getEME2000()          # Define ECI ref frame.
GCRF = FF.getGCRF()            # GCRF ref frame
ICRF = FF.getICRF()            # ICRF ref frame
ECEF = FF.getITRF(IERSConventions.IERS_2010, True)  # Define ECEF ref frame.
TEME = FF.getTEME()            # Define TEME ref frame. 
ITRF = ECEF                    # ITRF ref frame

R_earth  = Constants.WGS84_EARTH_EQUATORIAL_RADIUS             # Radius of earth
Mu_earth = Constants.WGS84_EARTH_MU                            # Gravitational parameter of earth
f_earth  = Constants.WGS84_EARTH_FLATTENING                    # Earth flattening value

earth1 = OneAxisEllipsoid(R_earth, f_earth, ITRF)               # Create earth here.
#display(earth0)

earth = ReferenceEllipsoid.getWgs84(ITRF) # import earth wgs84
#display(earth)

In [3]:
# initialize the objects in orekit
# initializing the ground station
lat_rad = radians(40.1164) # latitude N in radians
lon_rad = radians(-88.2434) # longitude E in radians
elevation = 233.0 #meters

ground_geo = GeodeticPoint(lat_rad, lon_rad, elevation) # ground in geo coords
# Frame of the ground station is given as Topocentric Frame.
ground_frame = TopocentricFrame(earth, ground_geo, "Champaign")
# Create the ground station object
ground_station = GroundStation(ground_frame)

# initializing the observable satellite object
obs_sat = ObservableSatellite(1) 
obs_sata = ObservableSatellite(2) 

In [4]:
# read observation data from csv file
df = pd.read_csv(r'AE502_HW4.csv')
data = df.to_numpy()
# the mjd times are converted into orekit AbsoluteDate class
date = [AbsoluteDate(MJD, t.item()) for t in data[:,4]]
# convert other data to radians
ra = np.radians(data[:,0].T).tolist()
dec = np.radians(data[:,1].T).tolist()
alt = np.radians(data[:,2].T).tolist()
azi = np.radians(data[:,3].T).tolist()

sigma  = [1.0, 1.0] # given uncertainty in HW
baseWeight  = [1.0, 1.0] # baseWeight

In [5]:
i=0
# use orekit to observe and estimate position of the satellite from RA and Declination
raDec0 = AngularRaDec(ground_station, ECI, date[i], [ra[i], dec[i]], 
                      sigma, baseWeight, obs_sat)
# direction vector from RA and Declination - Line of Sight
LOS0   = np.array([[cos(raDec0.getObservedValue()[1]) * cos(raDec0.getObservedValue()[0])],
                   [cos(raDec0.getObservedValue()[1]) * sin(raDec0.getObservedValue()[0])],
                   [sin(raDec0.getObservedValue()[1])]])
ground_station0_ECI = ground_frame.getPVCoordinates(date[i], ECI).getPosition()    
pos0 = np.array([[ground_station0_ECI.getX()], 
                 [ground_station0_ECI.getY()], 
                 [ground_station0_ECI.getZ()]])

i = 1
raDec1 = AngularRaDec(ground_station, ECI, date[i], [ra[i], dec[i]], 
                      sigma, baseWeight, obs_sat)
LOS1   = np.array([[cos(raDec1.getObservedValue()[1]) * cos(raDec1.getObservedValue()[0])],
                   [cos(raDec1.getObservedValue()[1]) * sin(raDec1.getObservedValue()[0])],
                   [sin(raDec1.getObservedValue()[1])]])

ground_station1_ECI = ground_frame.getPVCoordinates(date[i], ECI).getPosition()    
pos1 = np.array([[ground_station1_ECI.getX()], 
                 [ground_station1_ECI.getY()], 
                 [ground_station1_ECI.getZ()]])

i = 2
raDec2 = AngularRaDec(ground_station, ECI, date[i], [ra[i], dec[i]], 
                      sigma, baseWeight, obs_sat)
LOS2   = np.array([[cos(raDec2.getObservedValue()[1]) * cos(raDec2.getObservedValue()[0])],
                   [cos(raDec2.getObservedValue()[1]) * sin(raDec2.getObservedValue()[0])],
                   [sin(raDec2.getObservedValue()[1])]])

ground_station2_ECI = ground_frame.getPVCoordinates(date[i], ECI).getPosition()    
pos2 = np.array([[ground_station2_ECI.getX()], 
                 [ground_station2_ECI.getY()], 
                 [ground_station2_ECI.getZ()]])

In [6]:
obs     = list([raDec0, raDec1, raDec2])
LOS     = np.hstack((LOS0, LOS1, LOS2))
LOS_inv = np.linalg.inv(LOS)
R_site  = np.hstack((pos0, pos1, pos2))

R_site_df = pd.DataFrame({'1st Observation': [R_site[0][0], R_site[1][0], R_site[2][0]],
                          '2nd Observation': [R_site[0][1], R_site[1][1], R_site[2][1]],
                          '3rd Observation': [R_site[0][2], R_site[1][2], R_site[2][2]]}, 
                           index = ['X (m)', 'Y (m)', 'Z (m)'])
R_site_df.index.name = 'R in ECI'
display(R_site_df)

tau1 = obs[0].getDate().durationFrom(obs[1].getDate())   # [sec]
tau3 = obs[2].getDate().durationFrom(obs[1].getDate())   # [sec]

a1   = tau3/(tau3 - tau1)
a1_u = tau3*((tau3 - tau1)**2 - tau3**2) / (6*(tau3 - tau1))
a3   = -tau1/(tau3 - tau1)
a3_u = -tau1*((tau3 - tau1)**2 - tau1**2) / (6*(tau3 - tau1))

M = np.matmul(LOS_inv, R_site)

d1 = M[1][0]*a1 - M[1][1] + M[1][2]*a3
d2 = M[1][0]*a1_u + M[1][2]*a3_u
C  = np.dot(LOS[:,1], R_site[:,1])

print("\nd1: ",d1)
print("d2: ",d2)
print("C: ",C)

,1st Observation,2nd Observation,3rd Observation
R in ECI,,,
X (m),-3.791703e+06,-3.791703e+06,-3.791702e+06
Y (m),-3.147307e+06,-3.147308e+06,-3.147308e+06
Z (m),4.035892e+06,4.035892e+06,4.035892e+06



d1:  0.009816810488700867
d2:  211.02863425173933
C:  4318738.054231258


In [7]:
#### Create an 8th degree polynomial in r_2
P = list(range(9))

P[0] = 1                                                   # 8th
P[1] = 0                                                   # 7th
P[2] = -(d1**2 + 2*C*d1 + np.linalg.norm(R_site[:,1])**2)  # 6th
P[3] = 0                                                   # 5th
P[4] = 0                                                   # 4th
P[5] = -2*Mu_earth*(C*d2 + d1*d2)                          # 3rd
P[6] = 0                                                   # 2nd
P[7] = 0                                                   # 1st
P[8] = -Mu_earth**2 * d2**2                                # 0th

roots = np.roots(P)

PosRoots = []
for item in roots:
    if np.isreal(item) and item > 0:
        PosRoots.append(item)

if len(PosRoots) == 1:
    r2 = PosRoots[0].real
else:
    pass

# Below value allows you to find an initial estimate of the slant-range values, 
# which ends Gauss’s part of the solution. First, find the three c_i coefficients.
u = Mu_earth / r2**3

c1 = -(-a1 -a1_u*u)
c2 = -1
c3 = -(-a3 -a3_u*u)

# Now, determine the initial guess of the slant ranges. Be sure to multiply the matrices out first.
B = np.matmul(M, np.array([[-c1], [-c2], [-c3]]))

A = np.identity(3)
A[0,0] = c1
A[1,1] = c2
A[2,2] = c3

slant_ranges = np.linalg.solve(A,B)
slant_ranges_df = pd.DataFrame(slant_ranges, 
                               index = ['1st Observation', '2nd Observation', '3rd Observation'])
slant_ranges_df.columns = ['Slant Range (m)']
display(slant_ranges_df)

position = []
for i in range(3):
    pos = slant_ranges[i]*LOS[:, i] + R_site[:, i]
    pos = np.transpose(pos) 
    position = np.concatenate((position, pos))

position = position.reshape((3, 3))
position_df = pd.DataFrame({'1st Observation': [position[0][0], position[1][0], position[2][0]],
                            '2nd Observation': [position[0][1], position[1][1], position[2][1]],
                            '3rd Observation': [position[0][2], position[1][2], position[2][2]]}, 
                            index = ['X (m)', 'Y (m)', 'Z (m)'])

display(position_df)

,Slant Range (m)
1st Observation,0.009281
2nd Observation,0.010142
3rd Observation,0.011986


,1st Observation,2nd Observation,3rd Observation
X (m),-3.791703e+06,-3.147307e+06,4.035892e+06
Y (m),-3.791703e+06,-3.147308e+06,4.035892e+06
Z (m),-3.791702e+06,-3.147308e+06,4.035892e+06


In [20]:
# These values can be refined. 
# First Curtis's algorithm is used to find velocity vector at t=2

f1 = 1 - (0.5*(Mu_earth/np.linalg.norm(position[1,:])**3)*tau1**2)
f3 = 1 - (0.5*(Mu_earth/np.linalg.norm(position[1,:])**3)*tau3**2)
g1 = tau1 - ((1/6)*(Mu_earth/np.linalg.norm(position[1,:])**3)*tau1**3)
g3 = tau3 - ((1/6)*(Mu_earth/np.linalg.norm(position[1,:])**3)*tau3**3)

v2 = (1/(f1*g3 - f3*g1))*(-f3*position[0,:] + f1*position[2,:])

# Position vector p2 is taken from the result of Gauss IOD.
p2 = Vector3D(float(position[1,0]), float(position[1,1]), float(position[1,2]))
v2 = Vector3D(float(v2[0]), float(v2[1]), float(v2[2]))
PVcoord = PVCoordinates(p2, v2)

cartesianOrbit = CartesianOrbit(PVcoord, ECI, date[1], Mu_earth)
keplerElements = OrbitType.KEPLERIAN.convertType(SpacecraftState(cartesianOrbit, 1.0).getOrbit())
display(cartesianOrbit)
print('\033[1m'+'\nFor second observation: \n'+'\033[0m', keplerElements)

<CartesianOrbit: Cartesian parameters: {P(-3791702.652045044, -3147307.5694101173, 4035891.9529665383), V(231.31027880193744, -272.0521344763609, 4.3459947664687295)}>


For second observation: 
 Keplerian parameters: {a: 3188016.135898227; e: 0.9979620158035872; i: 39.32802907247767; pa: 268.8167419721716; raan: 131.22373310934282; v: -179.99983086636283;}


In [18]:
display(df)


,Right_Ascension_deg,Declination_deg,Altitude_deg,Azimuth_deg,time_MJD
0,265.386952,38.453957,19.354708,55.548143,60055.134028
1,286.015716,47.780985,14.270320,38.235439,60055.135417
2,306.835887,50.451438,8.135668,25.945747,60055.136805
3,113.107229,76.196138,40.470905,341.811449,60055.210417
4,44.452179,71.688355,23.635703,350.782760,60055.211806
5,27.541026,61.873318,12.451733,355.010106,60055.213194
6,128.555554,21.405043,10.129990,289.423811,60055.285417
7,116.989445,32.831985,9.416887,305.296788,60055.286805
8,102.466965,41.066986,6.877328,319.746407,60055.288194


In [14]:
# Using Laplace Initial Orbit Determination

Laplace = IodLaplace(Mu_earth)

# ECI Frame is the Earth Centered Inertial Frame. In Orekit the EME2000 or J2000 Frame is included. 
raDec_i = [AngularRaDec(ground_station, ECI, date[i], [ra[i], dec[i]], sigma, baseWeight, obs_sat) for i in range(9)]


estimated_orbit = Laplace.estimate(ECI, ground_frame.getPVCoordinates(date[1], ECI), raDec_i[0], raDec_i[1],raDec_i[2])


keplerElements = OrbitType.KEPLERIAN.convertType(SpacecraftState(estimated_orbit, 1.0).getOrbit())

print('\033[1m'+'\nFor second observation: \n'+'\033[0m', keplerElements)
print('\033[1m'+' \n'+'\033[0m', estimated_orbit)


For second observation: 
 Keplerian parameters: {a: 3187999.8174669943; e: 0.9979722364335792; i: 39.322593288194454; pa: 269.2003479546651; raan: 130.72805319189277; v: -179.99999762241717;}
 
 Cartesian parameters: {P(-3791702.6538640177, -3147307.563073164, 4035891.945700487), V(229.4927874741561, -272.4316352434614, 3.145754785356098)}


In [15]:
estimated_orbit = Laplace.estimate(ECI, ground_frame.getPVCoordinates(date[4], ECI), raDec_i[3], raDec_i[4],raDec_i[5])

keplerElements = OrbitType.KEPLERIAN.convertType(SpacecraftState(estimated_orbit, 1.0).getOrbit())

print('\033[1m'+'\nFor 5th observation: \n'+'\033[0m', keplerElements)
print('\033[1m'+' \n'+'\033[0m', estimated_orbit)


For 5th observation: 
 Keplerian parameters: {a: 3188001.193995367; e: 0.997971373845824; i: 39.3229321357896; pa: 269.17145468115694; raan: 130.76569482161716; v: -179.99997873433855;}
 
 Cartesian parameters: {P(-3791685.127726954, -3147328.373563607, 4035892.1828947756), V(229.64811458012116, -272.3989119069349, 3.22360912319916)}


In [12]:
estimated_orbit = Laplace.estimate(ECI, ground_frame.getPVCoordinates(date[7], ECI), raDec_i[6], raDec_i[7],raDec_i[8])

keplerElements = OrbitType.KEPLERIAN.convertType(SpacecraftState(estimated_orbit, 1.0).getOrbit())

print('\033[1m'+'\nFor 8th observation: \n'+'\033[0m', keplerElements)
print('\033[1m'+' \n'+'\033[0m', estimated_orbit)


For second observation: 
 Keplerian parameters: {a: 3188000.0875931955; e: 0.9979720671189193; i: 39.32272927731812; pa: 269.1890092194566; raan: 130.74333502348676; v: -179.99999571105204;}
 
 Cartesian parameters: {P(-3791667.9207076607, -3147348.805753795, 4035892.4146077093), V(229.53655807445753, -272.4137282393766, 3.186852519898903)}


In [16]:
from org.hipparchus.linear import CholeskyDecomposer, LUDecomposer, QRDecomposer
from org.hipparchus.optim.nonlinear.vector.leastsquares import SequentialGaussNewtonOptimizer, GaussNewtonOptimizer
from org.orekit.estimation.leastsquares import SequentialBatchLSEstimator, BatchLSEstimator
from org.orekit.propagation.analytical import KeplerianPropagator


Decomp = CholeskyDecomposer(1.0, 1.0)
KepProp = KeplerianPropagator(keplerElements, Mu_earth)
display(KepProp.getInitialState())

KepProp.propagate(date[1],date[2])

display(KepProp)

#GNO = GaussNewtonOptimizer(Decomp, True)

#LSEst = BatchLSEstimator(GNO, KepProp)

#LSEst.addMeasurement(raDec0)


<SpacecraftState: SpacecraftState{orbit=Keplerian parameters: {a: 3188001.193995367; e: 0.997971373845824; i: 39.3229321357896; pa: 269.17145468115694; raan: 130.76569482161716; v: -179.99997873433855;}, attitude=org.orekit.attitudes.Attitude@2eb79cbe, mass=1000.0, additional={}, additionalDot={}}>

<KeplerianPropagator: org.orekit.propagation.analytical.KeplerianPropagator@6b3871d6>